***test d ablation pour le model de darija bert "SI2M-Lab/DarijaBERT"***

1- Retirer la régularisation

In [1]:
!pip install transformers==4.26.0


In [3]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
from sklearn.model_selection import train_test_split

# Lire le fichier depuis Drive (modifie le chemin si tu l’as mis ailleurs)
df = pd.read_csv('/content/drive/MyDrive/darija_dataset_fusionne_cleaned.csv')
df = df[["clean_text", "final_label"]].dropna()

# Mapping des labels
label2id = {"negative": 0, "neutral": 1, "positive": 2}
id2label = {v: k for k, v in label2id.items()}
df["label"] = df["final_label"].map(label2id)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["clean_text"].tolist(), df["label"].tolist(), test_size=0.2, random_state=42
)
from transformers import AutoTokenizer

model_name = "SI2M-Lab/DarijaBERT"
tokenizer = AutoTokenizer.from_pretrained(model_name)

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)
import torch

class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()} | {'labels': torch.tensor(self.labels[idx])}
    def __len__(self):
        return len(self.labels)

train_dataset = TweetDataset(train_encodings, train_labels)
val_dataset = TweetDataset(val_encodings, val_labels)
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3,
    id2label=id2label,
    label2id=label2id
)
import os
os.environ["WANDB_DISABLED"] = "true"

from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score

training_args_no_wd = TrainingArguments(
    output_dir="./results_no_wd",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.0,  # ABLATION : pas de regularisation
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_dir="./logs_no_wd",
)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = predictions.argmax(-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted"),
    }

# Initialisation du Trainer
trainer = Trainer(
    model=model,
    args=training_args_no_wd,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Entraînement
trainer.train()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
Some weights of the model checkpoint at SI2M-Lab/DarijaBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassificati

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.862200,0.776533,0.649639,0.645432
2,0.639700,0.725760,0.690909,0.689645
3,0.453600,0.712254,0.734199,0.734437
4,0.364800,0.755666,0.746320,0.747126


***** Running Evaluation *****
  Num examples = 3465
  Batch size = 16
Saving model checkpoint to ./results_no_wd/checkpoint-866
Configuration saved in ./results_no_wd/checkpoint-866/config.json
Model weights saved in ./results_no_wd/checkpoint-866/pytorch_model.bin
tokenizer config file saved in ./results_no_wd/checkpoint-866/tokenizer_config.json
Special tokens file saved in ./results_no_wd/checkpoint-866/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 3465
  Batch size = 16
Saving model checkpoint to ./results_no_wd/checkpoint-1732
Configuration saved in ./results_no_wd/checkpoint-1732/config.json
Model weights saved in ./results_no_wd/checkpoint-1732/pytorch_model.bin
tokenizer config file saved in ./results_no_wd/checkpoint-1732/tokenizer_config.json
Special tokens file saved in ./results_no_wd/checkpoint-1732/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 3465
  Batch size = 16
Saving model checkpoint to ./results_no_wd/checkpoint-

TrainOutput(global_step=3464, training_loss=0.5633951676084593, metrics={'train_runtime': 1363.516, 'train_samples_per_second': 40.648, 'train_steps_per_second': 2.54, 'total_flos': 3645699516039168.0, 'train_loss': 0.5633951676084593, 'epoch': 4.0})

In [4]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 3465
  Batch size = 16


{'eval_loss': 0.7556663155555725,
 'eval_accuracy': 0.7463203463203463,
 'eval_f1': 0.7471256645027478,
 'eval_runtime': 23.6773,
 'eval_samples_per_second': 146.343,
 'eval_steps_per_second': 9.165,
 'epoch': 4.0}

2-augmentation du Taille du batch a 32

In [5]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
from sklearn.model_selection import train_test_split

# Lire le fichier depuis Drive (modifie le chemin si tu l’as mis ailleurs)
df = pd.read_csv('/content/drive/MyDrive/darija_dataset_fusionne_cleaned.csv')
df = df[["clean_text", "final_label"]].dropna()

# Mapping des labels
label2id = {"negative": 0, "neutral": 1, "positive": 2}
id2label = {v: k for k, v in label2id.items()}
df["label"] = df["final_label"].map(label2id)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["clean_text"].tolist(), df["label"].tolist(), test_size=0.2, random_state=42
)
from transformers import AutoTokenizer

model_name = "SI2M-Lab/DarijaBERT"
tokenizer = AutoTokenizer.from_pretrained(model_name)

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)
import torch

class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()} | {'labels': torch.tensor(self.labels[idx])}
    def __len__(self):
        return len(self.labels)

train_dataset = TweetDataset(train_encodings, train_labels)
val_dataset = TweetDataset(val_encodings, val_labels)
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3,
    id2label=id2label,
    label2id=label2id
)
import os
os.environ["WANDB_DISABLED"] = "true"

from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score

training_args_no_wd = TrainingArguments(
    output_dir="./results_no_wd",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=4,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_dir="./logs_no_wd",
)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = predictions.argmax(-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted"),
    }

# Initialisation du Trainer
trainer = Trainer(
    model=model,
    args=training_args_no_wd,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Entraînement
trainer.train()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--SI2M-Lab--DarijaBERT/snapshots/4025e5465fc6b3c9aa516839dd0bd039fa2e6d99/config.json
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
Model config BertConfig {
  "_name_or_path": "SI2M-Lab/DarijaBERT",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "cls_token": "[CLS]",
  "do_lower_case": true,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.834730,0.612987,0.603929
2,0.831200,0.715366,0.692352,0.691634
3,0.632200,0.702773,0.711977,0.712598
4,0.488300,0.719366,0.724098,0.724160


***** Running Evaluation *****
  Num examples = 3465
  Batch size = 32
Saving model checkpoint to ./results_no_wd/checkpoint-433
Configuration saved in ./results_no_wd/checkpoint-433/config.json
Model weights saved in ./results_no_wd/checkpoint-433/pytorch_model.bin
tokenizer config file saved in ./results_no_wd/checkpoint-433/tokenizer_config.json
Special tokens file saved in ./results_no_wd/checkpoint-433/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 3465
  Batch size = 32
Saving model checkpoint to ./results_no_wd/checkpoint-866
Configuration saved in ./results_no_wd/checkpoint-866/config.json
Model weights saved in ./results_no_wd/checkpoint-866/pytorch_model.bin
tokenizer config file saved in ./results_no_wd/checkpoint-866/tokenizer_config.json
Special tokens file saved in ./results_no_wd/checkpoint-866/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 3465
  Batch size = 32
Saving model checkpoint to ./results_no_wd/checkpoint-1299


TrainOutput(global_step=1732, training_loss=0.619036394784412, metrics={'train_runtime': 1288.9961, 'train_samples_per_second': 42.998, 'train_steps_per_second': 1.344, 'total_flos': 3645699516039168.0, 'train_loss': 0.619036394784412, 'epoch': 4.0})

In [6]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 3465
  Batch size = 32


{'eval_loss': 0.719365656375885,
 'eval_accuracy': 0.7240981240981241,
 'eval_f1': 0.7241602709914315,
 'eval_runtime': 21.8086,
 'eval_samples_per_second': 158.882,
 'eval_steps_per_second': 4.998,
 'epoch': 4.0}

3- L'augmentation de max_length à 256

In [7]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
from sklearn.model_selection import train_test_split

# Lire le fichier depuis Drive (modifie le chemin si tu l’as mis ailleurs)
df = pd.read_csv('/content/drive/MyDrive/darija_dataset_fusionne_cleaned.csv')
df = df[["clean_text", "final_label"]].dropna()

# Mapping des labels
label2id = {"negative": 0, "neutral": 1, "positive": 2}
id2label = {v: k for k, v in label2id.items()}
df["label"] = df["final_label"].map(label2id)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["clean_text"].tolist(), df["label"].tolist(), test_size=0.2, random_state=42
)
from transformers import AutoTokenizer

model_name = "SI2M-Lab/DarijaBERT"
tokenizer = AutoTokenizer.from_pretrained(model_name)

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=256)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=256)
import torch

class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()} | {'labels': torch.tensor(self.labels[idx])}
    def __len__(self):
        return len(self.labels)

train_dataset = TweetDataset(train_encodings, train_labels)
val_dataset = TweetDataset(val_encodings, val_labels)
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3,
    id2label=id2label,
    label2id=label2id
)
import os
os.environ["WANDB_DISABLED"] = "true"

from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score

training_args_no_wd = TrainingArguments(
    output_dir="./results_no_wd",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_dir="./logs_no_wd",
)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = predictions.argmax(-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted"),
    }

# Initialisation du Trainer
trainer = Trainer(
    model=model,
    args=training_args_no_wd,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Entraînement
trainer.train()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--SI2M-Lab--DarijaBERT/snapshots/4025e5465fc6b3c9aa516839dd0bd039fa2e6d99/config.json


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
Model config BertConfig {
  "_name_or_path": "SI2M-Lab/DarijaBERT",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "cls_token": "[CLS]",
  "do_lower_case": true,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "mask_token": "[MASK]",
  "max_len": 128,
  "max_position_embeddings": 512,
  "model_max_length": 128,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token": "[PAD]",
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "sep_token": "[SEP]",
  "transformers_version": "4.26.0",
  "type_vocab_size": 2,
  "unk_token": "[UNK]",
  "use_cache": true,
  "vocab_size": 80000
}


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.868100,0.754167,0.667244,0.666564
2,0.635600,0.715249,0.695527,0.692802
3,0.449400,0.712793,0.729870,0.729787
4,0.355300,0.748391,0.754401,0.754715


***** Running Evaluation *****
  Num examples = 3465
  Batch size = 16
Saving model checkpoint to ./results_no_wd/checkpoint-866
Configuration saved in ./results_no_wd/checkpoint-866/config.json
Model weights saved in ./results_no_wd/checkpoint-866/pytorch_model.bin
tokenizer config file saved in ./results_no_wd/checkpoint-866/tokenizer_config.json
Special tokens file saved in ./results_no_wd/checkpoint-866/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 3465
  Batch size = 16
Saving model checkpoint to ./results_no_wd/checkpoint-1732
Configuration saved in ./results_no_wd/checkpoint-1732/config.json
Model weights saved in ./results_no_wd/checkpoint-1732/pytorch_model.bin
tokenizer config file saved in ./results_no_wd/checkpoint-1732/tokenizer_config.json
Special tokens file saved in ./results_no_wd/checkpoint-1732/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 3465
  Batch size = 16
Saving model checkpoint to ./results_no_wd/checkpoint-

TrainOutput(global_step=3464, training_loss=0.5604712417990175, metrics={'train_runtime': 1618.6603, 'train_samples_per_second': 34.241, 'train_steps_per_second': 2.14, 'total_flos': 4215340065420288.0, 'train_loss': 0.5604712417990175, 'epoch': 4.0})

In [8]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 3465
  Batch size = 16


{'eval_loss': 0.7483914494514465,
 'eval_accuracy': 0.7544011544011544,
 'eval_f1': 0.7547152792640055,
 'eval_runtime': 22.6255,
 'eval_samples_per_second': 153.145,
 'eval_steps_per_second': 9.591,
 'epoch': 4.0}

***test d ablation pour le model de darija bert "SI2M-Lab/DarijaBERT-arabizi"***

1- retirer la regularisation

In [10]:
!pip install datasets evaluate
import pandas as pd
import numpy as np
import evaluate
import os
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    EvalPrediction,
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.5 MB/s eta 0:00:00


In [12]:
#Désactiver Weights & Biases
os.environ["WANDB_DISABLED"] = "true"
from google.colab import drive
drive.mount('/content/drive')
#Charger le dataset
df = pd.read_csv("/content/drive/MyDrive/darija_dataset_fusionne_cleaned.csv")

#Garder les commentaires en alphabet latin
df_latin = df[df['source_langue'] == 'latin']

#Renommer les colonnes
df_latin = df_latin.rename(columns={"commentaire": "clean_text", "sentiment": "final_label"})

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
#Split train/test
train_df, test_df = train_test_split(
    df_latin,
    test_size=0.2,
    random_state=42,
    stratify=df_latin["final_label"]
)

#Conversion en datasets Hugging Face
dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [15]:
#Tokenizer & modèle
model_name = "SI2M-Lab/DarijaBERT-arabizi"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

#Tokenisation
def tokenize_function(example):
    return tokenizer(example["clean_text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--SI2M-Lab--DarijaBERT-arabizi/snapshots/399963e234e368ed8428f32bf040e4a6e03c819c/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--SI2M-Lab--DarijaBERT-arabizi/snapshots/399963e234e368ed8428f32bf040e4a6e03c819c/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--SI2M-Lab--DarijaBERT-arabizi/snapshots/399963e234e368ed8428f32bf040e4a6e03c819c/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--SI2M-Lab--DarijaBERT-arabizi/snapshots/399963e234e368ed8428f32bf040e4a6e03c819c/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--SI2M-Lab--DarijaBERT-arabizi/snapshots/399963e234e368ed8428f32bf040e4a6e03c819c/config.json
Model config BertConfig {
  "_name_or_path": "SI2M

Map:   0%|          | 0/7452 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/1864 [00:00<?, ? examples/s]

In [16]:
#Encoder les labels
label2id = {"negative": 0, "neutral": 1, "positive": 2}

def encode_labels(example):
    example["labels"] = label2id[example["final_label"]]
    return example

tokenized_datasets = tokenized_datasets.map(encode_labels)
tokenized_test = tokenized_test.map(encode_labels)

Map:   0%|          | 0/7452 [00:00<?, ? examples/s]

Map:   0%|          | 0/1864 [00:00<?, ? examples/s]

In [17]:
#Supprimer les colonnes inutiles
cols_to_remove = ['source_langue', 'final_label', '__index_level_0__']
for col in cols_to_remove:
    if col in tokenized_datasets.column_names:
        tokenized_datasets = tokenized_datasets.remove_columns(col)
    if col in tokenized_test.column_names:
        tokenized_test = tokenized_test.remove_columns(col)

#Ajouter les mappings de labels au modèle (optionnel mais utile)
model.config.label2id = label2id
model.config.id2label = {v: k for k, v in label2id.items()}

#Métrique d'évaluation
accuracy = evaluate.load("accuracy")

def compute_metrics(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    return accuracy.compute(predictions=preds, references=p.label_ids)

In [21]:
#Paramètres d'entraînement
training_args = TrainingArguments(
    output_dir="./darijabert_arabizi_finetuned",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.0, # ABLATION : pas de regularisation
    logging_dir='./logs',
    logging_steps=10,
    save_total_limit=2,
    report_to="none",  # <-- pour ne plus voir le warning sur wandb
)

#Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

PyTorch: setting up devices


In [22]:
#Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

#Entraînement
trainer.train()


The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: clean_text, pays. If clean_text, pays are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 7452
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1864
  Number of trainable parameters = 170523651


Epoch,Training Loss,Validation Loss,Accuracy
1,0.070600,0.796160,0.833691
2,0.093000,0.862758,0.841738
3,0.064200,0.925226,0.839592
4,0.212400,1.053865,0.835300


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: clean_text, pays. If clean_text, pays are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1864
  Batch size = 8
Saving model checkpoint to ./darijabert_arabizi_finetuned/checkpoint-466
Configuration saved in ./darijabert_arabizi_finetuned/checkpoint-466/config.json
Model weights saved in ./darijabert_arabizi_finetuned/checkpoint-466/pytorch_model.bin
tokenizer config file saved in ./darijabert_arabizi_finetuned/checkpoint-466/tokenizer_config.json
Special tokens file saved in ./darijabert_arabizi_finetuned/checkpoint-466/special_tokens_map.json
Deleting older checkpoint [darijabert_arabizi_finetuned/checkpoint-1398] due to args.save_total_limit
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassifi

TrainOutput(global_step=1864, training_loss=0.0989303621893165, metrics={'train_runtime': 363.9552, 'train_samples_per_second': 81.9, 'train_steps_per_second': 5.122, 'total_flos': 530333871460848.0, 'train_loss': 0.0989303621893165, 'epoch': 4.0})

In [23]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: clean_text, pays. If clean_text, pays are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1864
  Batch size = 8


{'eval_loss': 1.0538650751113892,
 'eval_accuracy': 0.8353004291845494,
 'eval_runtime': 3.9284,
 'eval_samples_per_second': 474.488,
 'eval_steps_per_second': 59.311,
 'epoch': 4.0}

2-augmentation du Taille du batch a 32

In [24]:
#Paramètres d'entraînement
training_args = TrainingArguments(
    output_dir="./darijabert_arabizi_finetuned",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,# ABLATION : augmentation a 32
    num_train_epochs=4,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    save_total_limit=2,
    report_to="none",  # <-- pour ne plus voir le warning sur wandb
)

#Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

PyTorch: setting up devices


In [25]:
#Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

#Entraînement
trainer.train()


The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: clean_text, pays. If clean_text, pays are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 7452
  Num Epochs = 4
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 932
  Number of trainable parameters = 170523651


Epoch,Training Loss,Validation Loss,Accuracy
1,0.186300,0.869538,0.834764
2,0.084700,0.878623,0.844421
3,0.059200,1.009582,0.833155
4,0.080300,1.013889,0.834227


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: clean_text, pays. If clean_text, pays are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1864
  Batch size = 8
Saving model checkpoint to ./darijabert_arabizi_finetuned/checkpoint-233
Configuration saved in ./darijabert_arabizi_finetuned/checkpoint-233/config.json
Model weights saved in ./darijabert_arabizi_finetuned/checkpoint-233/pytorch_model.bin
tokenizer config file saved in ./darijabert_arabizi_finetuned/checkpoint-233/tokenizer_config.json
Special tokens file saved in ./darijabert_arabizi_finetuned/checkpoint-233/special_tokens_map.json
Deleting older checkpoint [darijabert_arabizi_finetuned/checkpoint-1398] due to args.save_total_limit
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassifi

TrainOutput(global_step=932, training_loss=0.07561492385110886, metrics={'train_runtime': 335.1292, 'train_samples_per_second': 88.945, 'train_steps_per_second': 2.781, 'total_flos': 593339257138824.0, 'train_loss': 0.07561492385110886, 'epoch': 4.0})

In [26]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: clean_text, pays. If clean_text, pays are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1864
  Batch size = 8


{'eval_loss': 1.0138894319534302,
 'eval_accuracy': 0.8342274678111588,
 'eval_runtime': 6.6111,
 'eval_samples_per_second': 281.951,
 'eval_steps_per_second': 35.244,
 'epoch': 4.0}

3- L'augmentation de max_length à 256

In [27]:
# Tokenizer & modèle
model_name = "SI2M-Lab/DarijaBERT-arabizi"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# Tokenisation avec max_length = 256
def tokenize_function(example):
    return tokenizer(
        example["clean_text"],
        padding="max_length",
        truncation=True,
        max_length=256  # <-- Augmentation ici
    )

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--SI2M-Lab--DarijaBERT-arabizi/snapshots/399963e234e368ed8428f32bf040e4a6e03c819c/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--SI2M-Lab--DarijaBERT-arabizi/snapshots/399963e234e368ed8428f32bf040e4a6e03c819c/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--SI2M-Lab--DarijaBERT-arabizi/snapshots/399963e234e368ed8428f32bf040e4a6e03c819c/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--SI2M-Lab--DarijaBERT-arabizi/snapshots/399963e234e3

Map:   0%|          | 0/7452 [00:00<?, ? examples/s]

Map:   0%|          | 0/1864 [00:00<?, ? examples/s]

In [28]:
#Encoder les labels
label2id = {"negative": 0, "neutral": 1, "positive": 2}

def encode_labels(example):
    example["labels"] = label2id[example["final_label"]]
    return example

tokenized_datasets = tokenized_datasets.map(encode_labels)
tokenized_test = tokenized_test.map(encode_labels)

Map:   0%|          | 0/7452 [00:00<?, ? examples/s]

Map:   0%|          | 0/1864 [00:00<?, ? examples/s]

In [29]:
#Supprimer les colonnes inutiles
cols_to_remove = ['source_langue', 'final_label', '__index_level_0__']
for col in cols_to_remove:
    if col in tokenized_datasets.column_names:
        tokenized_datasets = tokenized_datasets.remove_columns(col)
    if col in tokenized_test.column_names:
        tokenized_test = tokenized_test.remove_columns(col)

#Ajouter les mappings de labels au modèle (optionnel mais utile)
model.config.label2id = label2id
model.config.id2label = {v: k for k, v in label2id.items()}

#Métrique d'évaluation
accuracy = evaluate.load("accuracy")

def compute_metrics(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    return accuracy.compute(predictions=preds, references=p.label_ids)

In [30]:
#Paramètres d'entraînement
training_args = TrainingArguments(
    output_dir="./darijabert_arabizi_finetuned",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.0, # ABLATION : pas de regularisation
    logging_dir='./logs',
    logging_steps=10,
    save_total_limit=2,
    report_to="none",  # <-- pour ne plus voir le warning sur wandb
)

#Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

PyTorch: setting up devices


In [31]:
#Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

#Entraînement
trainer.train()


The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: clean_text, pays. If clean_text, pays are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 7452
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1864
  Number of trainable parameters = 170523651
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is 

Epoch,Training Loss,Validation Loss,Accuracy
1,0.625800,0.610392,0.748391
2,0.363600,0.529968,0.813305
3,0.278700,0.635209,0.828863
4,0.228300,0.732156,0.832082


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: clean_text, pays. If clean_text, pays are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1864
  Batch size = 8
Saving model checkpoint to ./darijabert_arabizi_finetuned/checkpoint-466
Configuration saved in ./darijabert_arabizi_finetuned/checkpoint-466/config.json
Model weights saved in ./darijabert_arabizi_finetuned/checkpoint-466/pytorch_model.bin
tokenizer config file saved in ./darijabert_arabizi_finetuned/checkpoint-466/tokenizer_config.json
Special tokens file saved in ./darijabert_arabizi_finetuned/checkpoint-466/special_tokens_map.json
Deleting older checkpoint [darijabert_arabizi_finetuned/checkpoint-699] due to args.save_total_limit
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassific

TrainOutput(global_step=1864, training_loss=0.39380139905495704, metrics={'train_runtime': 1516.3844, 'train_samples_per_second': 19.657, 'train_steps_per_second': 1.229, 'total_flos': 3921442377818112.0, 'train_loss': 0.39380139905495704, 'epoch': 4.0})

In [32]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: clean_text, pays. If clean_text, pays are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1864
  Batch size = 8


{'eval_loss': 0.7321556210517883,
 'eval_accuracy': 0.8320815450643777,
 'eval_runtime': 28.2548,
 'eval_samples_per_second': 65.971,
 'eval_steps_per_second': 8.246,
 'epoch': 4.0}